<span type="title">流水线处理器</span> | <span type="update">2018-07-16</span> | <span type="version">1</span>

<span type="intro"><p type="card-text">本章主要介绍流水线处理器。</p></span>

# 流水线基本原理

传统MIPS的执行指令主要分为五步，取指（存储器取指令，更新PC）、译码（寄存器堆读寄存器值）、执行（运算指令使用ALU，访存指令计算存储器地址）、访存（Load指令从储存器读数据，Store数据写入储存器），回写（将数据写入寄存器堆）。

![](w6p1.png)

如图的蓝色部分就是流水线寄存器所在位置，当添加流水线寄存器之后，在五个阶段进行的中间数据就得以保存，这样的话，当取指完成其功能后就进行下一条指令，**节省了时间，提高了时间连续性和吞吐率。**

如果一条指令需要1000ps，每个部分需要200ps，那么对于单周期处理器而言，时钟周期为1000ps，对于流水线处理器而言，时钟周期为200ps。

**那么，可以无限的划分流程，而无限的提高频率吗？**实际情况是，因为流水寄存器的消耗，每个部分会出现延迟，当划分段数越多，延迟越多。但是因为划分段数越多，指令频率越高，账面越好看，但是实际执行时间会随着每部分流水线组件交接的延时而拖累整体的执行速度。

1993年的Pentium处理器流水线深度为5级，到了2004年，Pemtium4的深度为31级，这个时期完全是AMD和Intel为满足消费者对于时钟频率的需求而产生的畸形产品。1997年的ARM9为5级，到了2013年，Core i7稳定在14级/16级，而CortexA57则为15级。

**流水线的不平衡问题**：流水线的第二个问题是，如果每个阶段的运行时间不相等，那么就要以最慢的那个阶段为时钟频率。因此，尽可能的划分阶段，并且让其近似相等是有好处的。

# 超标量流水线

超标量流水线指的是在流水线的基础上，每条流水线都有自己的地址生成逻辑、ALU和数据Cache口，这样在一个时钟周期内可以同时发送两条指令。Pentium是第一款超标量的x86 CPU。

即两个ALU（双执行），两条指令输入（经过指令Cache和译码器后，指令生成），两条主存输入（双访存、双回写）。

超标量示意图如下所示：

![](w6p3.png)

超标量不一定要有流水，1964年第一台超级计算机CDC6600采用了10个并行功能部件，但是没有流水线。

**总结**

- 单周期→标量流水线：时间并行性的优化，主要是对现有硬件的切分
- 标量流水线→超标量流水线：空间并行性的优化，需成倍增加硬件资源

现代多核心CPU多是在一个CPU芯片组进城了多个超变量处理器核，这些CPU Core拥有自己的数据和指令缓存32kb左右，还有256kb的L2 Cache，多个CPU核共享8MB的L3 Cache缓存。

同时，因为x86指令太过复杂，为了使用流水线技术，x86内部指令都是由MIPS指令实现的。

# 流水线冒险

Hazard 冒险，指的是阻止下一条指令在下一个时钟周期开始执行的情况。其中分为三种：

- 结构冒险：所需的硬件部件在为之前的指令工作
- 数据冒险：需要等待之前的指令完成数据读写
- 控制冒险：需要根据之前的指令结果决定下一步的行动

## 数据冒险

如图所示：

![](w6p4.png)

add指令需要sub指令的t0寄存器内容，而t0寄存器在900ps之后才有数据。因此产生了数据冒险，解决办法之一是软件插入nop指令，跳过两次流水线周期，这样的话，如绿色箭头所示，就可以正常读写了。这又称之为流水线停顿stall，产生空泡bubble。

但是这样的效率不高，并且没有提供一个抽象的硬件接口，解决方案之二是数据前递（Forwarding）

![](w6p6.png)

因为我们并非真的要t0数据，而是需要在600ps后的ALU的数据，并且将其传入add指令的ALU中，因此，在这两者之间建立联系即可。

![](w6p5.png)

如上所示，前递又称旁路，为了解决此数据冒险，绿色的线段即可完成。同样的，为了解决紫色线段的问题，使用紫色的方案即可解决。

但是，如果一条指令需要使用之前指令的**访存**结果而不是运算结果呢？如红色所示。

![](w6p7.png)

这样的解决方案是，使用六实现停顿，并且数据前递到ALU中。

# 控制冒险

转义指令对流水线危害很大，因为我们在完整运行一条指令之前不知道下一条指令，因此，如下图所示，经过统计，每4-7条指令就会产生一条转移指令，其比例约为20%，对于AMD Athlon K7而言，转移指令损失10个周期以上。

![](w6p8.png)

对于以下三条转移指令：

- j Target 无条件直接转移
- beq t0 t1 Target 有条件直接转移
- jr t0 无条件间接转移

可以看到，转移的开销主要来自：转移条件判断引起的开销（直接转移）以及生成目标地址引起的开销（间接转移）

对于无条件直接转移，`j Target; New PC ={(PC+4)[31..28], address, 
00}`

Target中存放地址，此地址需要经过PC+4后连接，然后填充0之后形成新地址。

其所需的信息在取指阶段即可获得，流水线不会停顿。

对于无条件间接转移， `jr rs; New PC = R[rs]`

其所需信息在译码阶段才能获得转移目标的地址，流水线停顿1周期

对于有条件直接转移， `beq rs, rt, imm16` 这个更为复杂，需要在译码阶段比较寄存器内容，停顿1周期，在执行阶段完成转移条件判断，一共停顿2周期。因此解决方案是在寄存器堆外面添加额外的比较电路，这样就减少1周期消耗。

此外，还有一种叫做延迟转移技术，比如：

![](w6p9.png)

合理安排汇编代码顺序，可以避免转移指令的周期消耗。